In [2]:
import pickle
import pandas as pd
import cobra
from collections import defaultdict, OrderedDict, Counter
from cobra import Reaction, Metabolite, Model, Gene
import time
from glob import glob

from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.patches as mpatches
import sympy
from sympy import to_dnf, Add
from Bio import SeqIO
import re
import os
import scipy
import urllib
import scipy.stats as stats
from statsmodels.stats.multitest import *

%matplotlib inline

In [3]:
import sys
sys.path.append("/home/yara/Documents/PseudoFind")
from PseudoFind.pangenome_cmds import *

# Pseudomonas

In [4]:
# curate metadata

FT = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT.csv', dtype = {'Genome ID':str}).fillna('')

NCBI_FT = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/ncbiFT.csv')
NCBI_FT['Assembly'] = [x.strip() for x in NCBI_FT['Assembly']]
FT = FT.merge(NCBI_FT, left_on = 'Assembly Accession', right_on = 'Assembly', how = 'left')


FT['species'] = [x.split(' ')[1] for x in FT['Genome Name']]
FT = FT.loc[FT['species'] == 'aeruginosa']
FT = FT.loc[FT['Contigs'] < 150]
FT['Sequencing Depth'] = [str(x).lower().replace('x','').replace('nan','').replace(',','.') for x in FT['Sequencing Depth']]
FT = FT.loc[FT['Sequencing Depth'] != '']
FT['Sequencing Depth'] = [float(x) for x in FT['Sequencing Depth']]
FT = FT.loc[FT['Sequencing Depth'] > 10].fillna('')

LRI = ['Airway secretion', 'Cystic Fibrosis lung sputum', 'Endotracheal aspirate', 'Lung biopsy', 'Respiratory Sample','Sputum', 'bronchial ', 'bronchoalveolar ',
       'broncho alveolar', 'cystic fibrosis airway', 'cystic fibrosis sputum', 'endotracheal tube (ETT) aspirate', 'lung', 'lungs (tracheal aspirate)', 'nasopharynx', 
       'oropharynx, sputum, or bronchoalveolar lavage', 'patient with a chronic lung infection with P', 'patient with bronchiectasis', 'patient with pneumonia', 'pleural fluid',
      'pneumonia patient sputum', 'respiratory', 'sputum', 'throat of cystic fibrosis patient', 'throat', 'tracheal ', 'nasopharynx','bronchoalveolar ','sinus', 'Respiratory ',
       'Sputum ', 'Bronchiectasis ', 'Throat (pharynx)', 'Bronchial aspirate', 'Nasal mucosa', 'Nasopharynx', 'Tracheal ', ]
FT_LRI = FT.loc[[index for index in FT.index if any(y in FT.loc[index, 'Isolation Source']+' ' for y in LRI)]]


In [ ]:
# write code to scrape the metadata off from all of the NCBI_FTs first (and curate to get a more complete dataset?)

rows = {}
for index in FT.loc[FT['RefSeq FTP'] != ''].index:
    try:
        ftp_p = FT.loc[index, 'RefSeq FTP']
        ftp_id = ftp_p.split('/')[-1]
        print(ftp_id)
        to_retrieve = '%s/%s_genomic.gbff.gz'%(ftp_p, ftp_id)

        # download
        F_genome_url = urllib.request.urlopen(to_retrieve)
        with open('/home/yara/Downloads/%s.gz'%ftp_id, 'wb') as y:
            y.write(F_genome_url.read())

        # unzip
        os.system('gunzip %s'%('/home/yara/Downloads/%s.gz'%ftp_id))

        # scrape metadata
        for seqrecord in SeqIO.parse('/home/yara/Downloads/%s'%ftp_id, 'genbank'):
            break
        features = {x:y[0] for feature in seqrecord.features if feature.type == 'source' for x,y in feature.qualifiers.items()} 
        features.update({'title': [r1.title for r1 in seqrecord.annotations['references']][0], 'source': seqrecord.annotations['source']})
        rows[ftp_id] = features

        # clean up
        os.system('rm %s'%'/home/yara/Downloads/%s'%ftp_id)
    except:
        print(FT.loc[index, 'Genome ID'])

GCF_000284555.1_ASM28455v1
GCF_000296325.1_PAO579-1.0
GCF_000287875.1_AH16
GCF_000342145.1_PA_21_ST175v1
GCF_000359565.1_PA45
GCF_000414035.1_ASM41403v1
GCF_000468935.2_ASM46893v1
GCF_000473745.2_ASM47374v3
GCF_000481945.1_Pseu_aeru_CF127_V1
GCF_000481925.1_Pseu_aeru_CF18_V1
GCF_000481905.1_Pseu_aeru_CF27_V1
GCF_000481885.1_Pseu_aeru_CF5_V1
GCF_000481865.1_Pseu_aeru_X24509_V1
GCF_000481845.1_Pseu_aeru_UDL_V1
GCF_000481825.1_Pseu_aeru_S54485_V1
GCF_000481805.1_Pseu_aeru_JJ692_V1
GCF_000481785.1_Pseu_aeru_U2504_V1
GCF_000481765.1_Pseu_aeru_19660_V1
GCF_000481745.1_Pseu_aeru_6077_V1
GCF_000481725.1_Pseu_aeru_S35004_V1
GCF_000481705.1_Pseu_aeru_X13273_V1
GCF_000481685.1_Pseu_aeru_BWH001_V1
GCF_000481665.1_Pseu_aeru_BWH002_V1
GCF_000481645.1_Pseu_aeru_BWH003_V1
GCF_000481625.1_Pseu_aeru_BWH004_V1
GCF_000481585.1_Pseu_aeru_BWH006_V1
GCF_000481565.1_Pseu_aeru_BWH007_V1
GCF_000481545.1_Pseu_aeru_BWH008_V1
GCF_000481525.1_Pseu_aeru_BWH009_V1


In [52]:
# # download genomes

# output_dir = '/home/yara/Documents/cystic_fibrosis/data/pseudomonas/fna'
# list_of_gids = list(FT_LRI['Genome ID'])
# get_fna_PATRIC(list_of_gids, output_dir)

In [51]:
# # annotate with PROKKA
# focusing on 785 strains isolated from the lung/nose 434 of which were isolated from CF patients

list_of_fna_files = [x for x in glob('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/fna/*') if x.split('/')[-1].split('.fna')[0] in list(FT_LRI['Genome ID'])]
output_folder  = '/home/yara/Documents/cystic_fibrosis/data/pseudomonas/prokka'
prokka_annotate(list_of_fna_files, output_folder)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
512
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [10]:
# build allele matrix 

In [7]:
# fasta_files = glob('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/prokka/*/*.faa')

# filtered dataset : remove data from upper respiratory system
FT_LRI = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT_LRI.csv', dtype = {'Genome ID':str}, index_col = ['Unnamed: 0'])
to_filter = ['oropharynx, sputum, or bronchoalveolar lavage', 'nasopharynx', 'throat of cystic fibrosis patient','Throat', 'throat', 'nasopharynx', 'Nasopharynx', 'nasopharynx', 'sinus', 'Throat (pharynx)', 'Nasal mucosa']
FT_LRI = FT_LRI.reset_index().set_index('Isolation Source').drop(to_filter)
FT_LRI.reset_index().to_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT_LRI_f.csv')

fasta_files = [x for x in glob('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/prokka/*/*.faa') if x.split('/')[-1].split('.faa')[0] in list(FT_LRI['Genome ID'])]
output_directory = '/home/yara/Documents/cystic_fibrosis/data/pseudomonas/allele_matrix/'
get_allele_matrix(fasta_files, output_directory, return_allele_matrix = False)

0.000000% complete
0.140449% complete
0.280899% complete
0.421348% complete
0.561798% complete
0.702247% complete
0.842697% complete
0.983146% complete
1.123596% complete
1.264045% complete
1.404494% complete
1.544944% complete
1.685393% complete
1.825843% complete
1.966292% complete
2.106742% complete
2.247191% complete
2.387640% complete
2.528090% complete
2.668539% complete
2.808989% complete
2.949438% complete
3.089888% complete
3.230337% complete
3.370787% complete
3.511236% complete
3.651685% complete
3.792135% complete
3.932584% complete
4.073034% complete
4.213483% complete
4.353933% complete
4.494382% complete
4.634831% complete
4.775281% complete
4.915730% complete
5.056180% complete
5.196629% complete
5.337079% complete
5.477528% complete
5.617978% complete
5.758427% complete
5.898876% complete
6.039326% complete
6.179775% complete
6.320225% complete
6.460674% complete
6.601124% complete
6.741573% complete
6.882022% complete
7.022472% complete
7.162921% complete
7.303371% co

58.286517% complete
58.426966% complete
58.567416% complete
58.707865% complete
58.848315% complete
58.988764% complete
59.129213% complete
59.269663% complete
59.410112% complete
59.550562% complete
59.691011% complete
59.831461% complete
59.971910% complete
60.112360% complete
60.252809% complete
60.393258% complete
60.533708% complete
60.674157% complete
60.814607% complete
60.955056% complete
61.095506% complete
61.235955% complete
61.376404% complete
61.516854% complete
61.657303% complete
61.797753% complete
61.938202% complete
62.078652% complete
62.219101% complete
62.359551% complete
62.500000% complete
62.640449% complete
62.780899% complete
62.921348% complete
63.061798% complete
63.202247% complete
63.342697% complete
63.483146% complete
63.623596% complete
63.764045% complete
63.904494% complete
64.044944% complete
64.185393% complete
64.325843% complete
64.466292% complete
64.606742% complete
64.747191% complete
64.887640% complete
65.028090% complete
65.168539% complete


In [29]:
from PseudoFind.pangenome_cmds import *

def get_allele_matrix(sequence_files, output_directory, return_allele_matrix = True, file_type = 'fasta'):
    all_alleles = {}
    alleles_in_gids = defaultdict(dict)
    locus2allele = {}
    count = 0
    all_sequences = []
    iii = 0

    for sequence_file in sequence_files:
        print('%f%% complete'%(100*iii/len(sequence_files)))
        iii += 1
        gid = sequence_file.split('/')[-2]
        
        if file_type == 'fasta':
            with open(sequence_file, 'r') as f:
                string = f.read()
            gid_sequences = {''.join(x.split('\n')[1:]):x.split('\n')[0].replace('>','').split(' ')[0] for x in string.split('\n>')}
            
        if file_type == 'genbank':
            FT = get_FT_from_gb(sequence_file)
            gid_sequences = FT.set_index('Nucleotide_sequence')['locus_tag'].to_dict()
        
        new_sequences = set(gid_sequences.keys())-set(all_sequences)
        
        all_alleles.update(zip(new_sequences, ['Allele_%s'%str(i+count) for i in range(len(new_sequences))]))
        locus2allele.update({gene_id:all_alleles[seq] for seq,gene_id in gid_sequences.items() if seq in new_sequences})
        
        gid_alleles = [all_alleles[seq] for seq in gid_sequences.keys()]
        alleles_in_gids[gid] ={ID:1 for ID in gid_alleles}
        count += len(new_sequences)
        all_sequences = set(list(gid_sequences.keys())+list(all_sequences))

    allele_matrix = pd.DataFrame(alleles_in_gids)        
    allele_matrix = allele_matrix.fillna(0)

    sparse_allele_matrix = scipy.sparse.csc_matrix(np.array(allele_matrix))
    scipy.sparse.save_npz('%s/allele_matrix.npz'%output_directory, sparse_allele_matrix)
    col_rows = [allele_matrix.columns, allele_matrix.index]
    pickle.dump(col_rows, open('%s/col_rows.p'%output_directory, 'wb'))
    pickle.dump(all_alleles, open('%s/all_alleles_seqs.p'%output_directory, 'wb'))
    pickle.dump(locus2allele, open('%s/locus2allele.p'%output_directory, 'wb'))
    
    if return_allele_matrix == True:
        return allele_matrix

In [18]:
col_rows[0]

Index(['1208103.3', '1352355.3', '1402503.3', '1402512.3', '1402513.3',
       '1402520.3', '1402521.3', '1402522.3', '1402525.3', '1402528.3',
       ...
       '287.9347', '287.9376', '287.9846', '287.9847', '287.9848', '287.9849',
       '287.9851', '287.9875', '287.9882', '287.9892'],
      dtype='object', length=712)

In [3]:
FT_LRI = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT_LRI_f.csv', index_col = ['Unnamed: 0'], dtype = {'Genome ID':str})
# fasta_files = [x for x in glob('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/prokka/*/*.ffn') if x.split('/')[-1].split('.ffn')[0] in list(FT_LRI['Genome ID'])]
fasta_files = [x for x in glob('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/prokka/*/*.gb*') if x.split('/')[-1].split('.gb')[0] in list(FT_LRI['Genome ID'])]
output_directory = '/home/yara/Documents/cystic_fibrosis/data/pseudomonas/allele_matrix_na/'
# os.makedirs(output_directory)

get_allele_matrix(fasta_files, output_directory, return_allele_matrix = False, file_type= 'genbank')

0.000000% complete
0.140449% complete
0.280899% complete
0.421348% complete
0.561798% complete
0.702247% complete
0.842697% complete
0.983146% complete
1.123596% complete
1.264045% complete
1.404494% complete
1.544944% complete
1.685393% complete
1.825843% complete
1.966292% complete
2.106742% complete
2.247191% complete
2.387640% complete
2.528090% complete
2.668539% complete
2.808989% complete
2.949438% complete
3.089888% complete
3.230337% complete
3.370787% complete
3.511236% complete
3.651685% complete
3.792135% complete
3.932584% complete
4.073034% complete
4.213483% complete
4.353933% complete
4.494382% complete
4.634831% complete
4.775281% complete
4.915730% complete
5.056180% complete
5.196629% complete
5.337079% complete
5.477528% complete
5.617978% complete
5.758427% complete
5.898876% complete
6.039326% complete
6.179775% complete
6.320225% complete
6.460674% complete
6.601124% complete
6.741573% complete
6.882022% complete
7.022472% complete
7.162921% complete
7.303371% co

58.146067% complete
58.286517% complete
58.426966% complete
58.567416% complete
58.707865% complete
58.848315% complete
58.988764% complete
59.129213% complete
59.269663% complete
59.410112% complete
59.550562% complete
59.691011% complete
59.831461% complete
59.971910% complete
60.112360% complete
60.252809% complete
60.393258% complete
60.533708% complete
60.674157% complete
60.814607% complete
60.955056% complete
61.095506% complete
61.235955% complete
61.376404% complete
61.516854% complete
61.657303% complete
61.797753% complete
61.938202% complete
62.078652% complete
62.219101% complete
62.359551% complete
62.500000% complete
62.640449% complete
62.780899% complete
62.921348% complete
63.061798% complete
63.202247% complete
63.342697% complete
63.483146% complete
63.623596% complete
63.764045% complete
63.904494% complete
64.044944% complete
64.185393% complete
64.325843% complete
64.466292% complete
64.606742% complete
64.747191% complete
64.887640% complete
65.028090% complete


In [9]:
for fasta_file in fasta_files:
    FT = get_FT_from_gb(fasta_file)
    with open(fasta_file.replace('.gbk','.faa'),'r') as f:
        string = f.read()
    gene_seqs = {x.split('\n')[0] for x in string.split('\n>')}
    if len(gene_seqs) != FT.shape[0]:
        print('y')
        break

KeyboardInterrupt: 

In [20]:
col_rows = pickle.load(open('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/allele_matrix_na/col_rows.p', 'rb'))
col_rows[0][:5]

Index(['1208103.3', '1352355.3', '1402503.3', '1402512.3', '1402513.3'], dtype='object')

In [11]:
len(set(FT['translation'])), len(set(FT['Nucleotide_sequence']))

(6551, 6556)

# removing longitudinal strains and duplicate entries

In [116]:
info = pd.read_csv("/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT_LRI_f2.csv", dtype = {'Genome ID':str})

# longitudinal study 1: 
to_filterout = info.loc[(info['Study'] == 'Study_3') & (info['Collection Date'] != '1-Nov-2011')].index.tolist()
to_filterout += info.loc[(info['Study'] == 'Study_3') & (info['Collection Date'] == '1-Nov-2011')].index.tolist()[1:]

# longitudinal study 2:

to_filterout += [index for index in info.loc[(info['Study'] == 'Study_6')].index if '2014' not in info.loc[index, 'Collection Date']]
to_filterout += [index for index in info.loc[(info['Study'] == 'Study_6')].index if '2014'  in info.loc[index, 'Collection Date']][1:]

# longitudinal study 3:
# https://mra.asm.org/content/4/2/e00231-16/figures-only

Study_2_metadata = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/Study_2_metadata.csv')
info_m = info.loc[(info['Study'] == 'Study_2')]
info_m = info_m.reset_index()
info_m = info_m.merge(Study_2_metadata, left_on = 'GenBank Accessions', right_on = 'Accession no.')
info_m = info_m.sort_values(by = 'Yr isolated', ascending = False)
to_keep = info_m.drop_duplicates(subset = 'Patient', keep = 'first')
to_filterout += list(set(info_m['level_0']) - set(to_keep['level_0']))


# duplicate samples 1:
to_filterout += list(set(info.loc[(info['Study'] == 'Study_5')].index) - set(info.loc[(info['Study'] == 'Study_5')].drop_duplicates(subset = 'collection_date').index))

# duplicate samples 2:
# https://jcm.asm.org/content/57/6/e02019-18
info['strain_m'] = [str(x)[:-1] for x in info['strain']]
to_filterout += list(set(info.loc[(info['Study'] == 'Study_0')].index) - set(info.loc[(info['Study'] == 'Study_0')].drop_duplicates(subset = 'strain_m').index))
del info['strain_m']

# duplicate samples 3:
info['strain_m'] = [str(x).split('.')[0] for x in info['strain']]
to_filterout += list(set(info.loc[(info['Study'] == 'Study_1')].index) - set(info.loc[(info['Study'] == 'Study_1')].drop_duplicates(subset = 'strain_m').index))

# to_filterout = info.loc[info['Study'] != 'miscellaneous'].index.tolist()
info = info.drop(to_filterout)
info.to_csv('/home/yara/Documents/cystic_fibrosis/data/pseudomonas/FT_LRI_f3.csv')

# Staphylococcus

In [16]:
FT = pd.read_csv('/home/yara/Documents/staph_clinical_isolates/data/metadata/PATRIC_FT.csv', dtype = {'Genome ID':str},low_memory=False).fillna('').replace('missing','')
cf_info = [col for col in FT.columns if 'cystic' in ''.join({str(x) for x in FT[col]}).lower()]
cf_gids = defaultdict(dict)

for index in FT.index:
    for col in cf_info:
        if 'cystic' in FT.loc[index, col].lower():
            cf_gids[FT.loc[index, 'Genome ID']].update({col:FT.loc[index, col]})
            
CF_evidence = pd.DataFrame(cf_gids).T

SoI_FT = pd.read_csv('/home/yara/Documents/staph_clinical_isolates/data/metadata/SoI_v4.csv', dtype = {'Genome ID': str})
for soi in set(SoI_FT['Isolation Group']):
    a = np.zeros(len(SoI_FT))
    a[SoI_FT.loc[SoI_FT['Isolation Group'] == soi].index] = 1
    SoI_FT[soi] = a

# Genomes whose names begin with “Patient” were obtained from CF patients, those whose names begin with “Blood” or “Abscess” were obtained from non-CF isolates, and those whose names begin with “Saureus” are reference genomes   
c = 'population dynamics of s. aureus in cystic fibrosis patients to determine transmission events utilizing wgs. this study uses whole genome sequencing (wgs) to determine strain relatedness and assess population dynamics of staphylococcus aureus isolates from a cohort of cf patients as assessed by strain relatedness. 311 s. aureus isolates were collected from respiratory cultures of 115 cf patients during a 22 month study period.'
tmp = FT.set_index('Genome ID').loc[cf_gids]
tmp['Comments'] = [x.lower() for x in tmp['Comments']]
notcf = [x for x,y in tmp.loc[tmp.loc[tmp['Comments'] == c].index]['Strain'].items() if not y.startswith('CFS')]

SoI_FT = SoI_FT.set_index('Genome ID')
SoI_FT['cystic_fibrosis_status'] = [1 if index in set(cf_gids) - set(notcf) else 0 for index in SoI_FT.index]
SoI_FT = SoI_FT.reset_index()
lung_indices = SoI_FT.loc[SoI_FT['LRI'] == 1].index
SoI_FT = SoI_FT.loc[lung_indices]

SoI_FT.to_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv')

In [ ]:
['Publication','Comments',
 'Additional Metadata',]

In [110]:
# patric_FT = pd.read_csv('/home/yara/Documents/staph_clinical_isolates/data/metadata/PATRIC_FT.csv', low_memory = False, dtype = {'Genome ID': str})
# SoI_FT = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv', index_col = ['Unnamed: 0'])
# SoI_FT = SoI_FT.merge(patric_FT, on = 'Genome ID', how = 'left').fillna('')
SoI_FT.loc[SoI_FT['Additional Metadata'] != ''].shape

(456, 89)

In [ ]:
longitudinal_study1 = 'Population dynamics of S. aureus in Cystic Fibrosis patients to determine transmission events utilizing WGS. This study uses whole genome sequencing (WGS) to determine strain relatedness and assess population dynamics of Staphylococcus aureus isolates from a cohort of CF patients as assessed by strain relatedness. 311 S. aureus isolates were collected from respiratory cultures of 115 CF patients during a 22 month study period.'
potential_duplicates1 = 'In this study, we performed whole-genome sequencing of 184 Staphylococcus aureus isolates from 135 patients treated in different departments of an Italian paediatric hospital. Samples included both MRSA and MSSA isolated from different body sites. Overall, the selected 135 single-patient isolates belonged to 28 different sequence types (STs) and 41 spa-types, and 28.1% of the samples were positive for Panton-Valentine Leucocidin (PVL). We moreover tested the presence of a number of virulence and antibiotic resistance genes, which were conserved at different levels. We moreover highlighted the presence of a very specific ST1-SCCmecIV-t127 PVL- clone, and we reconstructed its timed phylogenetic tree by including available reference genomes to understand whether it was a newly arising clone specific of the hospital considered.'
potential_duplicates2 = 'The genetic disorder cystic fibrosis is a life-limiting condition affecting ~70,000 people worldwide. Targeted, early, treatment of the dominant infecting species, Pseudomonas aeruginosa, has improved patient outcomes, however there is concern that other species are now stepping in to take its place. In addition, the necessarily long-term antibiotic therapy received by these patients may be providing a suitable environment for the emergence of antibiotic resistance. To investigate these issues, we employed whole-genome sequencing of 28 non-Pseudomonas bacterial strains isolated from three paediatric patients.'


In [114]:
OrderedDict(sorted(Counter(SoI_FT['Comments']).items(), key = lambda a:a[1], reverse = True))

OrderedDict([('MRSA Surveillance II', 419),
             ('Population dynamics of S. aureus in Cystic Fibrosis patients to determine transmission events utilizing WGS. This study uses whole genome sequencing (WGS) to determine strain relatedness and assess population dynamics of Staphylococcus aureus isolates from a cohort of CF patients as assessed by strain relatedness. 311 S. aureus isolates were collected from respiratory cultures of 115 CF patients during a 22 month study period.',
              291),
             ('MRSA Orange County', 100),
             ('In this study, we performed whole-genome sequencing of 184 Staphylococcus aureus isolates from 135 patients treated in different departments of an Italian paediatric hospital. Samples included both MRSA and MSSA isolated from different body sites. Overall, the selected 135 single-patient isolates belonged to 28 different sequence types (STs) and 41 spa-types, and 28.1% of the samples were positive for Panton-Valentine Leucocidi

In [53]:
SoI_FT = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv')

fasta_files = [x.replace('_m.faa','.gbk').replace('.faa','.gbk') for x in SoI_FT['Directory']]
output_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix_na/'
os.makedirs(output_directory)

get_allele_matrix(fasta_files, output_directory, return_allele_matrix = False, file_type= 'genbank')

0.000000% complete
0.096339% complete
0.192678% complete
0.289017% complete
0.385356% complete
0.481696% complete
0.578035% complete
0.674374% complete
0.770713% complete
0.867052% complete
0.963391% complete
1.059730% complete
1.156069% complete
1.252408% complete
1.348748% complete
1.445087% complete
1.541426% complete
1.637765% complete
1.734104% complete
1.830443% complete
1.926782% complete
2.023121% complete
2.119461% complete
2.215800% complete
2.312139% complete
2.408478% complete
2.504817% complete
2.601156% complete
2.697495% complete
2.793834% complete
2.890173% complete
2.986513% complete
3.082852% complete
3.179191% complete
3.275530% complete
3.371869% complete
3.468208% complete
3.564547% complete
3.660886% complete
3.757225% complete
3.853565% complete
3.949904% complete
4.046243% complete
4.142582% complete
4.238921% complete
4.335260% complete
4.431599% complete
4.527938% complete
4.624277% complete
4.720617% complete
4.816956% complete
4.913295% complete
5.009634% co

39.980732% complete
40.077071% complete
40.173410% complete
40.269750% complete
40.366089% complete
40.462428% complete
40.558767% complete
40.655106% complete
40.751445% complete
40.847784% complete
40.944123% complete
41.040462% complete
41.136802% complete
41.233141% complete
41.329480% complete
41.425819% complete
41.522158% complete
41.618497% complete
41.714836% complete
41.811175% complete
41.907514% complete
42.003854% complete
42.100193% complete
42.196532% complete
42.292871% complete
42.389210% complete
42.485549% complete
42.581888% complete
42.678227% complete
42.774566% complete
42.870906% complete
42.967245% complete
43.063584% complete
43.159923% complete
43.256262% complete
43.352601% complete
43.448940% complete
43.545279% complete
43.641618% complete
43.737958% complete
43.834297% complete
43.930636% complete
44.026975% complete
44.123314% complete
44.219653% complete
44.315992% complete
44.412331% complete
44.508671% complete
44.605010% complete
44.701349% complete


79.479769% complete
79.576108% complete
79.672447% complete
79.768786% complete
79.865125% complete
79.961464% complete
80.057803% complete
80.154143% complete
80.250482% complete
80.346821% complete
80.443160% complete
80.539499% complete
80.635838% complete
80.732177% complete
80.828516% complete
80.924855% complete
81.021195% complete
81.117534% complete
81.213873% complete
81.310212% complete
81.406551% complete
81.502890% complete
81.599229% complete
81.695568% complete
81.791908% complete
81.888247% complete
81.984586% complete
82.080925% complete
82.177264% complete
82.273603% complete
82.369942% complete
82.466281% complete
82.562620% complete
82.658960% complete
82.755299% complete
82.851638% complete
82.947977% complete
83.044316% complete
83.140655% complete
83.236994% complete
83.333333% complete
83.429672% complete
83.526012% complete
83.622351% complete
83.718690% complete
83.815029% complete
83.911368% complete
84.007707% complete
84.104046% complete
84.200385% complete


In [23]:
fasta_files = SoI_FT['Directory']
output_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
get_allele_matrix(fasta_files, output_directory, return_allele_matrix = False)

0.000000% complete
0.096339% complete
0.192678% complete
0.289017% complete
0.385356% complete
0.481696% complete
0.578035% complete
0.674374% complete
0.770713% complete
0.867052% complete
0.963391% complete
1.059730% complete
1.156069% complete
1.252408% complete
1.348748% complete
1.445087% complete
1.541426% complete
1.637765% complete
1.734104% complete
1.830443% complete
1.926782% complete
2.023121% complete
2.119461% complete
2.215800% complete
2.312139% complete
2.408478% complete
2.504817% complete
2.601156% complete
2.697495% complete
2.793834% complete
2.890173% complete
2.986513% complete
3.082852% complete
3.179191% complete
3.275530% complete
3.371869% complete
3.468208% complete
3.564547% complete
3.660886% complete
3.757225% complete
3.853565% complete
3.949904% complete
4.046243% complete
4.142582% complete
4.238921% complete
4.335260% complete
4.431599% complete
4.527938% complete
4.624277% complete
4.720617% complete
4.816956% complete
4.913295% complete
5.009634% co

40.173410% complete
40.269750% complete
40.366089% complete
40.462428% complete
40.558767% complete
40.655106% complete
40.751445% complete
40.847784% complete
40.944123% complete
41.040462% complete
41.136802% complete
41.233141% complete
41.329480% complete
41.425819% complete
41.522158% complete
41.618497% complete
41.714836% complete
41.811175% complete
41.907514% complete
42.003854% complete
42.100193% complete
42.196532% complete
42.292871% complete
42.389210% complete
42.485549% complete
42.581888% complete
42.678227% complete
42.774566% complete
42.870906% complete
42.967245% complete
43.063584% complete
43.159923% complete
43.256262% complete
43.352601% complete
43.448940% complete
43.545279% complete
43.641618% complete
43.737958% complete
43.834297% complete
43.930636% complete
44.026975% complete
44.123314% complete
44.219653% complete
44.315992% complete
44.412331% complete
44.508671% complete
44.605010% complete
44.701349% complete
44.797688% complete
44.894027% complete


80.154143% complete
80.250482% complete
80.346821% complete
80.443160% complete
80.539499% complete
80.635838% complete
80.732177% complete
80.828516% complete
80.924855% complete
81.021195% complete
81.117534% complete
81.213873% complete
81.310212% complete
81.406551% complete
81.502890% complete
81.599229% complete
81.695568% complete
81.791908% complete
81.888247% complete
81.984586% complete
82.080925% complete
82.177264% complete
82.273603% complete
82.369942% complete
82.466281% complete
82.562620% complete
82.658960% complete
82.755299% complete
82.851638% complete
82.947977% complete
83.044316% complete
83.140655% complete
83.236994% complete
83.333333% complete
83.429672% complete
83.526012% complete
83.622351% complete
83.718690% complete
83.815029% complete
83.911368% complete
84.007707% complete
84.104046% complete
84.200385% complete
84.296724% complete
84.393064% complete
84.489403% complete
84.585742% complete
84.682081% complete
84.778420% complete
84.874759% complete


In [ ]:
fasta_files = SoI_FT['Directory']
output_directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
get_allele_matrix(fasta_files, output_directory, return_allele_matrix = False)

In [4]:
# get pan genome and pan allelome curves

directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
allele_matrix_SA = scipy.sparse.load_npz('%s/allele_matrix.npz'%directory).todense().T
# col_rows = pickle.load(open('%s/col_rows.p'%directory, 'rb'))
# input_table = pd.DataFrame(allele_matrix_SA, index = col_rows[0], columns = col_rows[1]).T

In [41]:
# gid_list = [x for x in range(allele_matrix_SA.shape[0])]
# np.random.shuffle(gid_list)

# core = set()
# pan = []
# count = 0

# for gid in gid_list:
#     if core == set():
#         core = set(np.argwhere(allele_matrix_SA[gid,:] == 1)[:,1])
#     else:
#         core = set(np.argwhere(allele_matrix_SA[gid,:] == 1)[:,1]) & set(core)
#     print(len(core))  
#     if pan == []:
#         pan = np.argwhere(allele_matrix_SA[gid,:] == 1)[:,1]
#     else:
#         pan = np.union1d(np.argwhere(allele_matrix_SA[gid,:] == 1)[:,1], pan)

2576
2207
2136
1860
1785
1716
799
436
433
432
281
278
276
276
275
268
268
267
264
254
214
214
213
204
203
203
202
199
162
162
159
158
158
157
153
151
151
140
140
140
140
132
130
129
120
119
119
119
119
119
119
119
119
118
116
115
113
112
112
112
111
107
106
105
104
104
104
104
104
104
103
97
97
97
96
95
94
94
94
94
91
91
91
90
89
85
83
83
83
82
81
81
81
80
80
77
77
77
77
77
77
77
77
77
77
77
76
75
75
75
74
74
71
71
70
69
69
69
69
69
69
69


/home/yara/.local/lib/python3.5/site-packages/ipykernel_launcher.py:14 DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


69
69
69
69
69
69
69
69
69
69
69
69
69
69
68
68
68
68
68
66
66
66
66
64
63
63
62
62
60
60
60
60
60
60
58
58
58
58
58
58
57
57
57
57
57
57
49
49
48
47
47
47
47
47
46
46
46
46
46
46
46
46
46
45
45
45
45
45
45
45
45
44
43
41
41
40
40
40
39
39
39
37
35
35
35
34
33
33
33
33
33
33
33
33
33
33
33
33
33
33
32
32
31
31
30
30
30
30
30
30
28
28
28
28
26
26
26
26
26
26
26
26
26
26
26
26
26
26
25
25
25
25
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
23
23
23
23
22
22
22
22
22
22
22
22
22
21
21
21
21
21
21
21
21
21
21
20
20
19
19
19
19
19
19
19
19
19
19
19
19
19
19
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
17
17
17
17
17
17
16
16
16
16
16
16
16
16
15
15
15
15
15
15
15
15
15
15
15
15
15
14
14
14
14
14
14
13
13
13
13
13
13
13
13
13
13
13
13
13
13
12
12
12
12
12
12
12
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
10
10
10
10
10
10
10
10
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
8
8
8
8
8


In [5]:
# # get pan genome and pan allelome curves

# directory = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/allele_matrix/'
# allele_matrix_SA = scipy.sparse.load_npz('%s/allele_matrix.npz'%directory).todense().T
# col_rows = pickle.load(open('%s/col_rows.p'%directory, 'rb'))
# input_table = pd.DataFrame(allele_matrix_SA, index = col_rows[0], columns = col_rows[1]).T

# bootstrap_no = 1000
# pangenome_res = get_pangenome_counts(input_table, bootstrap_no)

KeyboardInterrupt: 

In [ ]:
# overlay extracted features onto phylogenetic tree to select for alleles undergoing convergent evolution


In [ ]:
# build a sub-tree for one clonal complex/serotype to visualize the distribution of strains (are there subgroups for CF strains?)


# word clouds

In [121]:
os.makedirs('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata')

In [122]:
FT = pd.read_csv('/home/yara/Documents/staph_clinical_isolates/data/metadata/PATRIC_FT.csv', dtype = {'Genome ID':str},low_memory=False).fillna('').replace('missing','')
cols = ['Host Name', 'Isolation Source', 'Body Sample Site', 'Host Health']
FT[cols].to_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_before.csv')

In [125]:
FT = pd.read_csv('/home/yara/Documents/staph_clinical_isolates/data/metadata/PATRIC_FT.csv', dtype = {'Genome ID':str},low_memory=False).fillna('').replace('missing','')
SoI_FT = pd.read_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/FT_LRI.csv')
SoI_FT.merge(FT, on = 'Genome ID')[cols].to_csv('/home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_after.csv')

In [126]:
input_file = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_before.csv'
output_file = '/home/yara/Documents/cystic_fibrosis/output/Figures/cloudmetadata_before.png'
wordcloud = 'wordcloud_cli --text %s --imagefile %s'%(input_file, output_file)
wordcloud

'wordcloud_cli --text /home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_before.csv --imagefile /home/yara/Documents/cystic_fibrosis/output/Figures/cloudmetadata_before.png'

In [127]:
input_file = '/home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_after.csv'
output_file = '/home/yara/Documents/cystic_fibrosis/output/Figures/cloudmetadata_after.png'
wordcloud = 'wordcloud_cli --text %s --imagefile %s'%(input_file, output_file)
wordcloud

'wordcloud_cli --text /home/yara/Documents/cystic_fibrosis/data/staphylococcus/metadata/annotations_after.csv --imagefile /home/yara/Documents/cystic_fibrosis/output/Figures/cloudmetadata_after.png'